In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

### use the paraot model to generate

In [ ]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

''' 
uncomment to get reproducable paraphrase generations
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)
'''

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

phrases = ["Can you recommed some upscale restaurants in Newyork?",
           "What are the famous places we should not miss in Russia?"
]

for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase)
  for para_phrase in para_phrases:
   print(para_phrase)

In [ ]:
ss
with open('origin.txt','r') as f:
    ss = f.readlines()

In [ ]:
para = []
for s in ss:
    s = s.replace('\n','')
    temp = parrot.augment(input_phrase= s)
    if temp == None:
        continue
    for x in temp:
        para.append(x[0] + "|||" + s)

In [ ]:
with open('para.txt', 'w') as f: # para -> origin
    for line in para:
        f.write(f"{line}\n")

### use the pegasus model to generate

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
num_beams = 5
num_return_sequences = 5
pega = []
for s in ss:
    s = s.replace('\n','')
    temp = get_response(s,num_return_sequences,num_beams)
    if temp == None:
        continue
    if len(temp) == 0:
        continue
    for x in temp:
        para.append(x + "|||" + s)

In [ ]:
with open('pega.txt', 'w') as f: # para -> origin
    for line in pega:
        f.write(f"{line}\n")

### when it comes to train

In [ ]:
# raw = []
# origin = []
# with open('para.txt','r') as f: # also pega.txt
#     ss = f.readlines()
#     temp = ss.split('|||')
#     raw.append(temp[0])
#     origin.append(temp[1])

In [2]:
import benepar, spacy
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
doc = nlp('The time for action is now. It is never too late to do something.')
sent = list(doc.sents)[0]
print(sent._.parse_string)
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
print(sent._.labels)
# ('S',)
print(list(sent._.children)[0])

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.